In [1]:
import numpy as np

In [95]:
# dynamic programming? avoid brute force
def monotonic_digit_sequence(l,strict):
    """
    Build a dynamic programming table that gives the number of 
    nondecreasing digit sequences of up to length l.
    The output will be an array of shape (10,l), with the (i,j)
    entry gives the number of length-j nondecreasing digit sequences
    whose first digit is i. This array can be populated efficiently
    according to the formula:
        (i,j) = \sum_{i'\geq i}(i',j-1)
        (i,1) = 1
    If strict=True, instead calculate the number of increasing digit 
    sequences. The formula is similar, except sum over i'>i.
    """
    tab = np.zeros((10,l)).astype(int)
    tab[:,0] = 1
    for j in range(1,l):
        for i in range(10):
            if strict:
                tab[i,j] = tab[i+1:,j-1].sum()
            else:
                tab[i,j] = tab[i:,j-1].sum()
    return tab

def apply_lower_bound(bound,strict):
    """
    Find count of monotonic digit sequences above the given lower bound.
    """
    # convert int into list of digits
    bound_digits = [int(d) for d in str(bound)] 
    l = len(bound_digits)
    
    # create DP table for counts
    tab = monotonic_digit_sequence(l,strict) 
    
    # start with total number of allowable digit seqs of length l
    tot = tab[:,l-1].sum() 

    # take away digit seqs of length l that start too low
    tot -= tab[:bound_digits[0],l-1].sum() 
    
    # go through other digits, subtracting seqs whose initial digit is too low
    for j in range(l-1): 
        if strict:
            lower_digit = bound_digits[j]+1
        else:
            lower_digit = bound_digits[j]
        if bound_digits[j+1] - lower_digit < 0:
            return tot
        else:
            tot -= tab[lower_digit:bound_digits[j+1],l-j-2].sum()
    return tot

# less efficient brute-force method
def check_integer(q):
    digits = [int(d) for d in str(q)]
    repeat=False
    for i in range(len(digits)-1):
        if digits[i+1] < digits[i]:
            return False
        if digits[i+1]==digits[i]:
            repeat=True
    return repeat
def brute_force(lb,ub):
    tot = 0
    for i in range(ub-lb):
        q = lb + i
        if check_integer(q):
            tot += 1
    return tot

In [102]:
# part 1
lb = 357253 
ub = 892942

# efficient
t1 = (apply_lower_bound(lb,strict=False) - apply_lower_bound(lb,strict=True))
t2 = (apply_lower_bound(ub,strict=False) - apply_lower_bound(ub,strict=True))
a1 = t1-t2

# brute force
a2 = brute_force(lb,ub)

print("Both methods agree: {}".format(a1==a2))
print("Part 1 answer: {}".format(a1))

Both methods agree: True
Part 1 answer: 530


In [110]:
# part 2

# brute force for part 2
def check_integer2(q):
    digits = [int(d) for d in str(q)] + [-1] # add virtual last digit
    repeat=False
    for i in range(len(digits)-2):
        if digits[i+1] < digits[i]:
            return False
        if digits[i+1]==digits[i] and digits[i+2]!=digits[i] and digits[i-1]!=digits[i]:
            repeat=True
    return repeat
def brute_force2(lb,ub):
    tot = 0
    for i in range(ub-lb):
        q = lb + i
        if check_integer2(q):
            tot += 1
    return tot

print("Part 2 answer: {}".format(brute_force2(lb,ub)))

Part 2 answer: 324
